# Online Retail BG-NBD ve Gamma-Gamma ile CLTV Tahmini

## İş Problemi

- İngiltere merkezli perakende şirketi satış ve pazarlama faaliyetleri için roadmap belirlemek istemektedir. Şirketin orta uzun vadeli plan yapabilmesi için var olan müşterilerin gelecekte şirkete sağlayacakları potansiyel değerin tahmin edilmesi gerekmektedir.

## Veri Seti Hikayesi

- Online Retail II isimli veri seti İngiltere merkezli bir perakende şirketinin 01/12/2009 - 09/12/2011 tarihleri arasındaki online satış işlemlerini içeriyor. Şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır ve çoğu müşterisinin toptancı olduğu bilgisi mevcuttur.

## Veri Sözlüğü

  | Değişken      | Açıklaması        |
  | ------------- | ----------------- |
  | InvoiceNo     | Fatura Numarası   |
  | StockCode     | Ürün Kodu         |
  | Description   | Ürün İsmi         |
  | Quantity      | Ürün Adedi        |
  | InvoiceDate   | Fatura Tarihi     |
  | UnitPrice     | Fatura Fiyatı     |
  | CustomerID    | Müşteri Numarası  |
  | Country       | Ülke İsmi         |

In [1]:
# Kütüphane aktivasyonları
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import warnings
import matplotlib
import squarify
import plotly.graph_objects as go
from datetime import datetime
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [2]:
# Görünürlük ayarları
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 500)

In [3]:
# Verinin import edilmesi
def online_retail(path,sheetname):
    dataframe = pd.read_excel(path, sheet_name = sheetname)
    return dataframe

df_ = online_retail("online_retail_II.xlsx","Year 2010-2011")

In [4]:
# Verinin kopyasının alınması
df = df_.copy()

In [5]:
# Veriye genel bakış
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    
check_df(df,head=5)

##################### Shape #####################
(541910, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00  2.550    17850.000  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00  3.390    17850.000  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00  2.750    17850.000  United Kingdom
3  536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6 2010-12-01 08:26:00  3.390

In [6]:
# Veri ön işleme
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


def online_retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    return dataframe

df = online_retail_data_prep(df)

In [7]:
# Müşteri Yaşam Boyu Değeri hesaplamak için kullanılan fonksiyon
def create_cltv_c(dataframe, profit=0.10):
    # CLTV hesaplamaları için müşterileri tekilleştirme işlemleri:
    cltv_c = dataframe.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(), # İşlemlerin müşteri bazlı tekilleştirilmesi
                                                   'Quantity': lambda x: x.sum(), # İşlemlerin toplam niceliklerinin müşteri bazlı toplamı
                                                   'TotalPrice': lambda x: x.sum()}) # İşlemlerin toplam ücretinin müşteri bazlı tekilleştirilmesi
   
    # cltv_c kolonlarının kolon bazlı olarak isimlerinin değiştirilmesi
    cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
    # avg_order_value (Ortalama Sipariş Değeri = Toplam Fiyat / Toplam İşlem)
    cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
    # purchase_frequency (Satın Alma Sıklığı = Toplam İşlem / Toplam Müşteri Sayısı)
    cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]
    # repeat rate (Tekrarlanan Sipariş Oranı (Birden fazla alışveriş yapan müşteri sayısı / Tüm müşteriler))
    repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
    # churn rate (Müşteri Terk Oranı = 1 - Tekrarlanan Sipariş Oranı) 
    churn_rate = 1 - repeat_rate
    # profit_margin (Kâr Marjı = Toplam Fiyat * (profit))
    cltv_c['profit_margin'] = cltv_c['total_price'] * profit
    # Customer Value (Müşteri Değeri = Ortalama Sipariş Değeri * Satın Alma Sıklığı)
    cltv_c['customer_value'] = (cltv_c['avg_order_value'] * cltv_c["purchase_frequency"])
    # Customer Lifetime Value ((Customer Value/ Churn Rate)* Profit Margin)
    cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
    # Segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])
    print("#################### CLV Analysis ########################################")
    print(cltv_c.groupby("segment").agg(["mean", "count"]).T)
    print("###########################################################################")

    return cltv_c

clv = create_cltv_c(df)

#################### CLV Analysis ########################################
segment                         D        C        B         A
total_transaction  mean     1.270    1.855    3.501    10.459
                   count 1085.000 1084.000 1084.000  1085.000
total_unit         mean   116.656  278.221  636.912  3307.567
                   count 1085.000 1084.000 1084.000  1085.000
total_price        mean   175.420  456.043 1052.800  5882.376
                   count 1085.000 1084.000 1084.000  1085.000
avg_order_value    mean   151.552  300.896  393.571   610.424
                   count 1085.000 1084.000 1084.000  1085.000
purchase_frequency mean     0.000    0.000    0.001     0.002
                   count 1085.000 1084.000 1084.000  1085.000
profit_margin      mean    17.542   45.604  105.280   588.238
                   count 1085.000 1084.000 1084.000  1085.000
customer_value     mean     0.040    0.105    0.243     1.356
                   count 1085.000 1084.000 1084.000  1085

In [8]:
# Müşteri Yaşam Boyu Değeri Tahmini hesaplamak için kullanılan fonksiyon
def create_cltv_p(dataframe, month=3,week = 4):
    # df.InvoiceDate.max()/Timestamp('2011-12-09 12:50:00')
    today_date = dt.datetime(2011, 12, 11)

    cltv_df = dataframe.groupby('Customer ID').agg(
        {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days, # recency
                         lambda InvoiceDate: (today_date - InvoiceDate.min()).days], # T
         'Invoice': lambda Invoice: Invoice.nunique(), # frequency
         'TotalPrice': lambda TotalPrice: TotalPrice.sum()}) # monetary

    cltv_df.columns = cltv_df.columns.droplevel(0)
    cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
    cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
    cltv_df["recency"] = cltv_df["recency"] / 7 # haftalık
    cltv_df["T"] = cltv_df["T"] / 7 # haftalık

    # 2. BG-NBD Modelinin Kurulması
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T'])

    cltv_df["expected_purc_"+str(week)+"_week"] = bgf.predict(week,
                                                  cltv_df['frequency'],
                                                  cltv_df['recency'],
                                                  cltv_df['T'])


    # 3. GAMMA-GAMMA Modelinin Kurulması
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary'])
    cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                                 cltv_df['monetary'])

    # 4. BG-NBD ve GG modeli ile CLTV'nin hesaplanması.
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency'],
                                       cltv_df['T'],
                                       cltv_df['monetary'],
                                       time=month,  # 3 aylık
                                       freq="W",  # T'nin frekans bilgisi.
                                       discount_rate=0.01)

    cltv = cltv.reset_index()
    cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
    cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])
    print("#################### BG/NBD - Gamma Gamma CLTV Analysis ########################################")
    print(cltv_final.groupby("segment").agg({"count", "mean"}).T)
    print("###########################################################################")

    return cltv_final


cltv_final2 = create_cltv_p(df)

#################### BG/NBD - Gamma Gamma CLTV Analysis ########################################
segment                               D         C         B         A
Customer ID             mean  15558.476 15309.634 15352.819 14947.359
                        count   712.000   711.000   711.000   711.000
recency                 mean     22.074    30.670    29.515    31.411
                        count   712.000   711.000   711.000   711.000
T                       mean     40.465    38.111    34.812    34.484
                        count   712.000   711.000   711.000   711.000
frequency               mean      3.065     4.096     5.442    11.359
                        count   712.000   711.000   711.000   711.000
monetary                mean    183.963   271.695   373.443   659.859
                        count   712.000   711.000   711.000   711.000
expected_purc_4_week    mean      0.283     0.480     0.647     1.089
                        count   712.000   711.000   711.000   7